# Sustinable Azure - Time-Shifting

This interactive notebook will guide you how to use Azure Container App Job to schedule task with carbon awareness in mind.

## How-To start

* Ensure you have a Container App Job.

## Job configuration

You may start with a job and a cron configured

# Install modules

In [ ]:
# Install-Module -Name az -Verbose
# Install-Module -Name Az.App
# Install-module -Name CarbonAwareComputing.Cmdlets

# Load account settings from file & initialize

In [ ]:
$CurrentDir = Get-Location
$SettingsFile = "AccountSettings - Container App Jobs.json"

$AccountSettingsFile = Join-Path -Path $CurrentDir -ChildPath $SettingsFile
if (-not (Test-Path $AccountSettingsFile))
{
    Copy-Item "$AccountSettingsFile.example" $AccountSettingsFile -ErrorAction 'Stop'
}

$Settings = Get-Content $AccountSettingsFile | ConvertFrom-Json
$TenantId = $Settings.TenantId
$AccountId = $Settings.AccountId
$SubscriptionId = $Settings.SubscriptionId
$ExecutionLocation = $Settings.ExecutionLocation

$ResourceGroupName = $Settings.ResourceGroupName
$JobName = $Settings.JobName 

## Connect Azure

In [ ]:
Connect-AzAccount -DeviceCode -Tenant $TenantId -Subscription $SubscriptionId 
#Connect-AzAccount -Tenant $TenantId -AccountId $AccountId -Subscription $SubscriptionId
Set-AzContext -SubscriptionId $SubscriptionId

## Get the forecast for best execution time

In [ ]:
$now = Get-Date
$JobExecutionTime = Get-CarbonAwareExecutionTime -Location $ExecutionLocation -EarliestExecutionTime $now -LatestExecutionTime ($now).AddHours(10) -EstimatedExecutionDuration "00:20:00" -FallbackExecutionTime $now.AddMinutes(5)
$JobExecutionTime = $JobExecutionTime.AddMinutes(5) 
Write-Host "Best execution time for $ExecutionLocation is $JobExecutionTime"

In [ ]:
$cronExpression = "$($JobExecutionTime.Minute) $($JobExecutionTime.Hour) $($JobExecutionTime.Day) $($JobExecutionTime.Month) *"
$job = Get-AzContainerAppJob -Name $JobName -ResourceGroupName $ResourceGroupName -SubscriptionId $SubscriptionId
Update-AzContainerAppJob -Name $JobName -ResourceGroupName $ResourceGroupName -SubscriptionId $SubscriptionId `
    -ScheduleTriggerConfigCronExpression $cronExpression `
    -ConfigurationReplicaTimeout $job.ConfigurationReplicaTimeout
